Following https://cloud.google.com/vertex-ai/docs/tutorials/train-tensorflow-bigquery

This notebook doesn't do anything well.  
It does load data from BigQuery and creates a dataset that can be used in train a model.
The model training does not work but it's complaining about the inputs which means it is getting a dataset for the input.
The data is not being split into test and train, I'm using the same data for both data sets.  Needs to be done but this notebook was just testing loading data from BQ.

In [1]:
import os
import sys
import tensorflow as tf


In [2]:
BQ_SOURCE = "bq://moz-fx-data-shared-prod.mlhackweek_search_stable.action_v1"

In [3]:
import json
import numpy as np

from google.cloud import bigquery

# using query from dzeber to flatten table
query = '''
    with subset as (
    SELECT
        metrics.string.search_meta_search_text as search_text,
        metrics.datetime.search_meta_url_select_timestamp as ts,
        metrics.string.search_meta_session_id as session_id,
        metrics.labeled_counter.search_meta_position as position,
        metrics.labeled_string.search_meta_url as url,
        metrics.labeled_string.search_meta_hostname as hostname,
        metrics.labeled_string.search_meta_title as title,
        metrics.labeled_string.search_meta_short_description as descr,
        metrics.labeled_string.search_meta_preamble as preamble,
        metrics.labeled_boolean.search_meta_selected as selected
    FROM
        `moz-fx-data-shared-prod.mlhackweek_search_live.action_v1`
    WHERE
        DATE(submission_timestamp) >= DATE('2021-01-19')
        and metrics.string.search_meta_session_id is not NULL
),
zipped as (
    SELECT
        search_text,
        ts,
        session_id,
        array_agg(
            struct(
                pos.value as position,
                url[offset(pos_offset)].value as url,
                hostname[offset(pos_offset)].value as hostname,
                title[offset(pos_offset)].value as title,
                descr[offset(pos_offset)].value as descr,
                preamble[offset(pos_offset)].value as preamble,
                selected[offset(pos_offset)].value as selected
            )
        ) as arr,
        sum(cast(selected[offset(pos_offset)].value as int)) as n_selected
    FROM subset, unnest(position) as pos
    with offset as pos_offset
    GROUP BY search_text, ts, session_id
)
select
    search_text,
    ts,
    session_id,
    array_length(zipped.arr) as n_results,
    n_selected,
    arr.position,
    arr.url,
    arr.hostname,
    arr.title,
    arr.descr,
    arr.preamble,
    arr.selected
from zipped, zipped.arr
'''
         
bqclient = bigquery.Client()


# Download a table
def download_table(bq_table_uri: str):
    # Remove bq:// prefix if present
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix) :]

    table = bigquery.TableReference.from_string(bq_table_uri)
    rows = bqclient.query(query) 
    return rows.to_dataframe()


dataframe = download_table(BQ_SOURCE)
dataframe = dataframe[["n_results", "n_selected", "position"]]
# TODO simplifying to just numerical columns instead of proper feature engineering.
dataframe.head()

,n_results,n_selected,position
0,1,1,1
1,1,1,1
2,1,1,1
3,2,1,1
4,2,1,2


In [4]:
LABEL_COLUMN = "position"
def convert_dataframe_to_dataset(
    df_train,
    df_validation  
):
  
    #TODO properly split data.
    df_train_x, df_train_y = df_train, df_train.pop(LABEL_COLUMN)
    df_validation_x, df_validation_y = df_train_x, df_train_y

    y_train = np.asarray(df_train_y)
    y_validation = np.asarray(df_validation_y)

    # Convert to numpy representation
    x_train = np.asarray(df_train_x).astype("float32")
    x_test = np.asarray(df_validation_x).astype("float32")

    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset_validation = tf.data.Dataset.from_tensor_slices((x_test, y_validation))
    return (dataset_train, dataset_validation)



In [5]:
dataset_train, dataset_validation = convert_dataframe_to_dataset(dataframe, dataframe)

# Shuffle train set
dataset_train = dataset_train.shuffle(len(dataframe))

def create_model(num_features):
    # The model code is crap.  It just does something.
    Dense = tf.keras.layers.Dense
    model = tf.keras.Sequential(
        [
            Dense(
                100,
                activation=tf.nn.relu,
                kernel_initializer="uniform",
                input_dim=num_features,
            ),
            Dense(75, activation=tf.nn.relu),
            Dense(50, activation=tf.nn.relu),
            Dense(25, activation=tf.nn.relu),
            Dense(3, activation=tf.nn.softmax),
        ]
    )

    # Compile Keras model
    optimizer = tf.keras.optimizers.RMSprop(lr=0.001)
    model.compile(
        loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer
    )

    return model

# Create the model
model = create_model(num_features=dataset_train._flat_shapes[0].dims[0].value)
# Train the model
model.fit(dataset_train, epochs=1, validation_data=dataset_train)

tf.saved_model.save(model, 'gcs://glenda-bucket')


2021-09-29 22:31:15.566295: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200160000 Hz
2021-09-29 22:31:15.567026: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a121104570 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-29 22:31:15.567050: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-29 22:31:15.567228: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:801 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:791 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:784 run_step  **
        outputs = model.t